# Lab | Cleaning numerical data

For this lab, we will be using the dataset in the Customer Analysis Business Case. This dataset can be found in `files_for_lab` folder.

## Context

An auto insurance company has collected some data about its customers including their _demographics_, _education_, _employment_, _policy details_, _vehicle information_ on which insurance policy is, and _claim amounts_. You will help the senior management with some business questions that will help them to better understand their customers, improve their services, and improve profitability.

**Some business Objectives**:

- Retain customers,
- Analyze relevant customer data,
- Develop focused customer retention programs.

Based on the analysis, take targeted actions to increase profitable customer response, retention, and growth.

### Instructions

1. Import the necessary libraries.

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from imblearn.under_sampling import TomekLinks

In [57]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2. Load the `we_fn_use_c_marketing_customer_value_analysis.csv` into the variable `customer_df`.

In [58]:
customer_df = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

3. First look at its main features (`head`, `shape`, `info`).

In [59]:
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,Suburban,Married,69,32,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,Suburban,Single,94,13,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,Suburban,Married,108,18,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,Suburban,Married,106,18,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,Rural,Single,73,12,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [60]:
customer_df.shape

(9134, 24)

In [61]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

4. Rename the columns so they follow the _PEP8_ (snake case).

In [62]:
customer_df.columns = customer_df.columns.str.lower().str.replace(" ", "_")

In [63]:
customer_df.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

5. Change effective to date column to datetime format.

In [64]:
customer_df.effective_to_date = pd.to_datetime(customer_df.effective_to_date, errors='raise')

In [65]:
customer_df.effective_to_date.head()

0   2011-02-24
1   2011-01-31
2   2011-02-19
3   2011-01-20
4   2011-02-03
Name: effective_to_date, dtype: datetime64[ns]

6. Check `NaN` values per column.

In [66]:
percentages = {}

for col in customer_df.columns:
    nulls_percent = customer_df[col].isna().sum() * 100 / customer_df.shape[0]
    percentages[col] = nulls_percent

null_percentages = pd.DataFrame(percentages.values(), index=percentages.keys(), columns=['null_percentage'])

null_percentages # No NaN values

,null_percentage
customer,0.0
state,0.0
customer_lifetime_value,0.0
response,0.0
coverage,0.0
education,0.0
effective_to_date,0.0
employmentstatus,0.0
gender,0.0
income,0.0


7. Define a function that differentiates between continuous and discrete variables. **Hint**: A number of unique values might be useful. Store continuous data into a `continuous` variable and do the same for `discrete` and categorical.

In [67]:
def split_numericals(data, threshold=100):
    num = data.select_dtypes(np.number)
    cont_columns = []
    disc_columns = []
    for col in num.columns:
        if len(num[col].unique()) > threshold:
            cont_columns.append(col)
        else:
            disc_columns.append(col)
    num_cont = data.loc[:, cont_columns]
    num_disc = data.loc[:, disc_columns]
    return num_cont, num_disc

In [68]:
df_num_cont, df_num_disc = split_numericals(customer_df)

In [69]:
df_num_cont.head()

,customer_lifetime_value,income,monthly_premium_auto,total_claim_amount
0,2763.519279,56274,69,384.811147
1,6979.535903,0,94,1131.464935
2,12887.431650,48767,108,566.472247
3,7645.861827,0,106,529.881344
4,2813.692575,43836,73,138.130879


In [70]:
df_num_disc.head()

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,32,5,0,1
1,13,42,0,8
2,18,38,0,2
3,18,65,0,7
4,12,44,0,1


8. For the categorical data, check if there is some kind of text in a variable so we would need to clean it. **Hint**: Use the same method you used in step 7. Depending on the implementation, decide what to do with the variables you get.

In [71]:
cat = customer_df.select_dtypes(object)

for col in cat.columns:
    print(col, ":", cat[col].unique())

customer : ['BU79786' 'QZ44356' 'AI49188' ... 'TD14365' 'UP19263' 'Y167826']
state : ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response : ['No' 'Yes']
coverage : ['Basic' 'Extended' 'Premium']
education : ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
employmentstatus : ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
gender : ['F' 'M']
location_code : ['Suburban' 'Rural' 'Urban']
marital_status : ['Married' 'Single' 'Divorced']
policy_type : ['Corporate Auto' 'Personal Auto' 'Special Auto']
policy : ['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'
 'Corporate L1' 'Personal L2' 'Special L1' 'Special L3']
renew_offer_type : ['Offer1' 'Offer3' 'Offer2' 'Offer4']
sales_channel : ['Agent' 'Call Center' 'Web' 'Branch']
vehicle_class : ['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury SUV' 'Sports Car'
 'Luxury Car']
vehicle_size : ['Medsize' 'Small' 'Large']


In [72]:
# Remove "Offer" from renew_offer_type & convert to numerical data
customer_df.renew_offer_type = customer_df.renew_offer_type.str.replace("Offer", "")
customer_df.renew_offer_type = customer_df.renew_offer_type.apply(lambda x: int(x))
customer_df.renew_offer_type = pd.to_numeric(customer_df.renew_offer_type)

In [73]:
customer_df.renew_offer_type.describe()

count    9134.000000
mean        1.970221
std         1.007576
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         4.000000
Name: renew_offer_type, dtype: float64

9. Get categorical features.

In [74]:
cat = customer_df.select_dtypes(object)
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer          9134 non-null   object
 1   state             9134 non-null   object
 2   response          9134 non-null   object
 3   coverage          9134 non-null   object
 4   education         9134 non-null   object
 5   employmentstatus  9134 non-null   object
 6   gender            9134 non-null   object
 7   location_code     9134 non-null   object
 8   marital_status    9134 non-null   object
 9   policy_type       9134 non-null   object
 10  policy            9134 non-null   object
 11  sales_channel     9134 non-null   object
 12  vehicle_class     9134 non-null   object
 13  vehicle_size      9134 non-null   object
dtypes: object(14)
memory usage: 999.2+ KB


10. What should we do with the `customer_id` column?

In [75]:
# Set customer as the index
customer_df.set_index('customer', drop=True, inplace=True)